In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import openpyxl
import matplotlib.pyplot as plt
%matplotlib inline

### This script does the L30 cleaning of tha data, creating 19 new columns (L40) for the 'problematic' attributes, summing them up, and deleting the L30

In [4]:
city="ams"
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
# Paths for the Population Data --------------------------------------------------------------
#path to ancillary data folder
ancillary_data_folder_path = base_dir + "/data_prep/{}_Projectdata/AncillaryData".format(city)
ancillary_POPdata_folder_path = base_dir + "/data_prep/{}_Projectdata/PopData".format(city)
image_path= base_dir + "/data_prep/{}_Projectdata/PopData/images".format(city)
years_list=[1992,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018] #1992,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,

In [5]:
dictProblem = {'L40_COD' :['L30_BEL_COD','L10_COD', 'L10_COD_COD' ,'L30_ZAR_COD', 'L30_OTH_COD', ],
                'L40_DEU':['L30_BRD_DEU', 'L10_DEU', 'L30_DDDE'],
                'L40_KEN' : ['L30_UK_KEN', 'L10_KEN'],
                'L40_YUG': ['L30_YUG_YUG', 'L30_YUG'],
                'L40_YEM': ['L30_YEM_YEM', 'L10_YEM'],
                'L40_UKcol': ['L30_UK_BWA', 'L30_UK_BWI', 'L30_UK_BRN', 'L30_UK_ANT', 'L30_UK_BWI1', 'L30_UK_HND', 'L30_UK_BEI','L30_UK_LKA',
                            'L30_UK_GOK','L30_UK_MWI', 'L30_UK_SGP', 'L30_UK_TZA','L30_UK_ZAF','L30_UK_LSO','L30_UK_MYS','L30_UK_AFR','L30_UK_CMR','L30_UK_MYS.1',
                            'L30_BOTs_SKT', 'L30_BOTs_TDC', 'L10_IOT'], #'L30_UK_KEN','L30_UK_BWI', 'L30_UK_BWI1', 'L30_UK_BEI', 'L30_UK_GOK','L30_UK_LKA'
                'L40_BELcol': [ 'L30_BEL_RWA', 'L30_BEL_URN'],
                'L40_PRTcol': ['L30_PRT_MOZ', 'L30_PRT_AGO', 'L30_PRT_AGO.1'],
                'L40_NLDcol': ['L30_NLD_PNG', 'L30_NLD_DEI', 'L30_NLD_BES', 'L30_NLD_BES1'],
                'L40_FRAcol': ['L30_FRA_FRI', 'L30_FRA_FWA', 'L30_FRA_CMR', 'L30_FRA_DJI', 'L30_FRA_IND'],
                'L40_ESP' : ['L30_CAN_ESP', 'L10_ESP'],
                'L40_MMR' : ['L30_BUMM', 'L10_MMR'],
                'L40_CHN' : ['L30_TIB_CHN', 'L10_CHN'],
                'L40_SOM' : ['L10_SOM', 'L30_FRA_SOM', 'L30_ITA_SOM', 'L30_UK_SOM'],
                'L40_RUS' : ['L10_RUS', 'L30_RUS_RUS'],
                'L40_CPV' : ['L10_CPV.1', 'L10_CPV'],
                'L40_ARE' : ['L10_ARE.1', 'L10_ARE'],
                'L40_VCT' : ['L10_VCT.1', 'L10_VCT'], 
                'L40_OTH': ['L30_OTH', 'L30_DYBJ', 'L30_FDA_POL', 'L30_AEF_AEF', 'L30_FRA_COD', 'L30_GOA_GOA', 'L30_HAW_HAW', 'L30_CHI_CHI', 'L30_KOR_KOR', 'L30_LEE_LEE',
                                'L30_PRT_MAD', 'L30_SGP_MLK', 'L30_OMN_OMN', 'L30_NHB', 'L30_YMD', 'L30_NTH_RHO', 'L30_NTH_VNM', 'L30_unk', 'L30_AUT_HUN', 'L30_HVO',
                                'L30_PRT_AFR', 'L30_PRT_IND', 'L30_PRT_TLS', 'L30_RHO', 'L30_SRW_MYS', 'L30_SEY_SYC', 'L30_SIAM_THA', 'L30_URS', 'L30_ESP_ESH ', 'L30_TRA_JOR',
                                'L30_TCH', 'L30_ZAN_TZA', 'L30_STH_RHO', 'L30_STH_VNM', 'L30_DEU_NAM', 'L30_INT', 'L30_PNG_PNG', 'L30_ESP_GNQ', 'L30_WSM_WSM', 'L30_ARE_ARE',
                                'L30_PRT_AZO', 'L30_GEIC', 'L30_ESP_AFR', 'L30_PRT_GNB', 'L30_PAN_PCZ', 'L30_TAS_AUS', 'L30_SCG', 'L30_ANT_ANT', 'L30_DEU_OAFR', 'L30_DEU_WAFR',
                                'L30_XKX']
}

### 1. Sum countries based on above dictionary and drop these attibutes 

In [6]:
for year in years_list:
    pathI = ancillary_POPdata_folder_path + "/process01/{}.xlsx".format(year)
    dfI = pd.read_excel(pathI, header=0)
    ndf = dfI.iloc[: , 1:].drop(dfI.index[0]) #
    #print(ndf.head(2))
    for key in dictProblem:
        selectList = dictProblem['{}'.format(key)]
        #print(key, selectList)
        select = [x for x in ndf if x in selectList]
        ndf['{}'.format(key)] = ndf.loc[:, select].sum(axis=1)
        for i in selectList:
            if i in ndf.columns:
                ndf.drop('{}'.format(i), inplace=True, axis=1)
    print(ndf.head(2))
    l10 = [x for x in ndf.columns if x.startswith("L10_") ]
    l30 = [x for x in ndf.columns if x.startswith("L30_") ]
    l40 = [x for x in ndf.columns if x.startswith("L40_") ]
    if len(l30)!=0:
        sys.exit()
    print("------------------------------ Creating xlsx:{0} ------------------------------".format(year))
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/process02/{}.xlsx".format(year),  index = False,  header=True)
    # Convert the dataframe to an XlsxWriter Excel object.
    ndf.to_excel(writer, sheet_name='Sheet1')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    

   grid_id  L1_TOTALPOP  L2_CHILDREN  L3_STUDENTS  L4_MOBILE_ADULTS  \
1     4617            7            4            0                 1   
2     4808            5            0            0                 0   

   L5_NOT_MOBILE_ADULTS  L6_ELDERLY  L7_IMMIGRANTS  L8_EU_IMMIGRANTS  \
1                     1           1              0                 0   
2                     1           4              0                 0   

   L9_NONEU_IMMIGRANTS  ...  L40_FRAcol  L40_ESP  L40_MMR  L40_CHN  L40_SOM  \
1                    0  ...         0.0      0.0      0.0      0.0      0.0   
2                    0  ...         0.0      0.0      0.0      0.0      0.0   

   L40_RUS  L40_CPV  L40_ARE  L40_VCT  L40_OTH  
1      0.0      0.0      0.0      0.0      0.0  
2      0.0      0.0      0.0      0.0      0.0  

[2 rows x 203 columns]
------------------------------ Creating xlsx:1992 ------------------------------
   grid_id  L1_TOTALPOP  L2_CHILDREN  L3_STUDENTS  L4_MOBILE_ADULTS  \
1     48